In [113]:
import matplotlib.pyplot as plt
import math
import numpy as np 
import pandas as pd 
import io 
import requests
import datetime as dt


Viết Phương trình

In [114]:
def dx(cap_CO2_Air,cap_CO2_Top,MC_BlowAir,MC_ExtAir,MC_PadAir,MC_AirCan,MC_AirTop,MC_AirOut,MC_TopOut):
    MC_BlowAir=def_MC_BlowAir(Eta_HeatCO2,U_Blow,P_Blow,A_Flr)
    print(MC_BlowAir)
    MC_ExtAir=def_MC_ExtAir(U_ExtCO2,Phi_ExtCO2,A_Flr)
    
    MC_PadAir=def_MC_PadAir(U_Pad,Phi_Pad,A_Flr,CO2_Out,CO2_Air)

    f_ThScr=def_f_ThScr(U_ThScr,K_ThScr,Delta_T_AirTop,g,RO_Mean,Delta_RO)
    MC_AirTop=def_MC_AirTop(f_ThScr,CO2_Air,CO2_Top)

    ff_VentSide=def_f_VentRoofSide(C_d,A_Flr,U_Roof,0,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind) # A_Roof=0
    Eta_InsScr=def_Eta_InsScr(Zeta_InsScr)
    f_leakage=def_f_leakage(v_Wind,C_leakage)
    f_VentRoofSide=def_f_VentRoofSide(C_d,A_Flr,U_Roof,A_Roof,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind)
    f_VentForced=def_f_VentForced(Eta_InsScr,U_VentForced,Phi_VentForced,A_Flr)
    f_VentSide=def_f_VentSide(Eta_InsScr,ff_VentSide,f_leakage,U_ThScr,f_VentRoofSide,Eta_Side,Eta_SideThr)
    MC_AirOut=def_MC_AirOut(f_VentSide,f_VentForced,CO2_Air,CO2_Out)


    ff_VentRoof=def_ff_VentRoof(C_d,U_Roof,A_Roof,A_Flr,g,h_Roof,Delta_T_AirOut,T_mean,C_w,v_Wind)
    f_VentRoof=def_f_VentRoof(Eta_InsScr,ff_VentRoof,f_leakage,U_ThScr,f_VentRoofSide,Eta_Roof,Eta_RoofThr) 
    # can be changed
    MC_TopOut=def_MC_TopOut(f_VentRoof,CO2_Top,CO2_Out)
    
    h_C_Buf=def_h_C_Buf(C_Buf,C_Max_Buf)
    L=def_L(L0,K,LAI,m)
    fT=def_f(Hd,R,T,T0,S)
    KT=def_KT_WithLAI(LAI,K_T0,Ha,R,T,T0)
    PT_Max=def_PT_Max(KT,fT)
    P_Max_LT=def_PMax_LT(P_MLT,PT_Max,L,L_0_5)
    P=def_P(CO2_Air,CO2_0_5,Res,P_Max_LT)
    R_HOHAP=0.01*P
    MC_AirCan=def_MC_AirCan(M_CH2O,h_C_Buf,P,R_HOHAP)
    

    dx1=(MC_BlowAir+MC_ExtAir+MC_PadAir-MC_AirCan-MC_AirTop-MC_AirOut)/cap_CO2_Air
    dx2=(MC_AirTop-MC_TopOut)/cap_CO2_Top
    return [dx1,dx2]

In [115]:
def def_MC_BlowAir(Eta_HeatCO2,U_Blow,P_Blow,A_Flr):
    return Eta_HeatCO2*U_Blow*P_Blow/A_Flr

def def_MC_ExtAir(U_ExtCO2,Phi_ExtCO2,A_Flr):
    return U_ExtCO2*Phi_ExtCO2/A_Flr

def def_MC_PadAir(U_Pad,Phi_Pad,A_Flr,CO2_Out,CO2_Air):
    return U_Pad*Phi_Pad*(CO2_Out-CO2_Air)/A_Flr

def def_f_ThScr(U_ThScr,K_ThScr,Delta_T_AirTop,g,RO_Mean,Delta_RO):
    return U_ThScr*K_ThScr*((Delta_T_AirTop)**(2/3))+(1-U_ThScr)*(((g*(1-U_ThScr)/(2*RO_Mean))*Delta_RO)**(1/2))

def def_MC_AirTop(f_ThScr,CO2_Air,CO2_Top):
    return f_ThScr*(CO2_Air-CO2_Top)

def def_f_VentRoofSide(C_d,A_Flr,U_Roof,A_Roof,U_Side,A_Side,g,h_SideRoof,Delta_T_AirOut,T_mean,C_w,v_Wind):
    component1=C_d/A_Flr
    component2=((U_Roof**2)*(U_Side**2)*(A_Roof**2)*(A_Side**2))/((U_Roof**2)*(A_Roof**2)+(U_Side**2)*(A_Side**2))
    component3=(2*g*h_SideRoof*Delta_T_AirOut)/T_mean
    component4=(((U_Roof*A_Roof+U_Side*A_Side)/2)**2)*C_w*(v_Wind**2)
    return component1*((component2*component3+component4)**(1/2))

def def_Eta_InsScr(Zeta_InsScr):
    return Zeta_InsScr*(2-Zeta_InsScr)

def def_f_leakage(v_Wind,C_leakage):
    if (v_Wind<0.25):
        return 0.25*C_leakage
    else:
        return v_Wind*C_leakage

def def_f_VentForced(Eta_InsScr,U_VentForced,Phi_VentForced,A_Flr):
    return Eta_InsScr*U_VentForced*Phi_VentForced/A_Flr

def def_f_VentSide(Eta_InsScr,ff_VentSide,f_leakage,U_ThScr,f_VentRoofSide,Eta_Side,Eta_SideThr):
    if (Eta_Side>=Eta_SideThr):
        return Eta_InsScr*ff_VentSide+0.5*f_leakage
    else:
        return Eta_InsScr*(U_ThScr*ff_VentSide+(1-U_ThScr)*f_VentRoofSide*Eta_Side)+0.5*f_leakage

def def_MC_AirOut(f_VentSide,f_VentForced,CO2_Air,CO2_Out):
    return (f_VentSide+f_VentForced)*(CO2_Air-CO2_Out)

def def_ff_VentRoof(C_d,U_Roof,A_Roof,A_Flr,g,h_Roof,Delta_T_AirOut,T_mean,C_w,v_Wind):
    component1=C_d*U_Roof*A_Roof/(2*A_Flr)
    component2=(g*h_Roof*Delta_T_AirOut/(2*T_mean))+C_w*(v_Wind**2)
    return component1*(component2**(1/2))

def def_f_VentRoof(Eta_InsScr,ff_VentRoof,f_leakage,U_ThScr,f_VentRoofSide,Eta_Roof,Eta_RoofThr):
    if (Eta_Roof>=Eta_RoofThr):
        return Eta_InsScr*ff_VentRoof+0.5*f_leakage
    else:
        return Eta_InsScr*(U_ThScr*ff_VentRoof+(1-U_ThScr)*f_VentRoofSide*Eta_Roof)+0.5*f_leakage

def def_MC_TopOut(f_VentRoof,CO2_Top,CO2_Out):
    return f_VentRoof*(CO2_Top-CO2_Out)

def def_h_C_Buf(C_Buf,C_Max_Buf):
    if (C_Buf>C_Max_Buf):
        return 0
    else:
        return 1

def def_MC_AirCan(M_CH2O,h_C_Buf,P,R):
    return M_CH2O*h_C_Buf*(P-R)



Quang Hợp

In [116]:
def def_P(CO2_Air,CO2_0_5,Res,P_Max_LT):
    return ((CO2_Air+CO2_0_5+Res*P_Max_LT)-((CO2_Air+CO2_0_5+Res*P_Max_LT)**2-4*CO2_Air*Res*P_Max_LT))**(1/2)/(2*Res)
def def_K(K_T0,Ha,R,T,T0):
    return K_T0*math.exp((-Ha/R)*(1/T-1/T0))
def def_f(Hd,R,T,T0,S):
    component1=1+math.exp((-Hd/R)*(1/T0-1/(Hd/S)))
    component2=1+math.exp((-Hd/R)*(1/T-1/(Hd/S)))
    return component1/component2
def def_PT_Max(KT,fT):
    return KT*fT
def def_L(L0,K,LAI,m):
    return L0*(1-(K*math.exp(-K*LAI))/(1-m))
def def_KT_WithLAI(LAI,K_T0,Ha,R,T,T0):
    return LAI*K_T0*math.exp((-Ha/R)*(1/T-1/T0))
def def_PMax_LT(P_MLT,PT_Max,L,L_0_5):
    return P_MLT*PT_Max*L/(L+L_0_5)


Định nghĩa tham số, hằng số có sẵn:

Lấy dữ liệu cho các biến cần lấy dữ liệu

In [117]:
DATA1=pd.read_csv("https://raw.githubusercontent.com/CEAOD/Data/master/GH_Cucumber/AutonomousGreenhouseChallengeFirstEdition(2018)/Sonoma/Greenhouse_climate.csv")
DATA2=pd.read_csv("https://raw.githubusercontent.com/CEAOD/Data/master/GH_Cucumber/AutonomousGreenhouseChallengeFirstEdition(2018)/meteo.csv")
df1=pd.DataFrame(DATA1)
df2=pd.DataFrame(DATA2)
df1

,AssimLight,BlackScr,CO2air,EnScr,GHtime,HumDef,PipeGrow,PipeLow,RHair,Tair,VentLee,Ventwind
0,NaN,NaN,NaN,NaN,43326.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,43326.003472,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,427.000000,0.0,43326.006944,3.34,0.0,0.0,80.4,19.9,52.900000,2.9
3,0.0,0.0,443.000000,0.0,43326.010417,3.36,0.0,0.0,80.4,19.9,52.900000,2.9
4,0.0,0.0,443.999999,0.0,43326.013889,3.29,0.0,0.0,80.9,19.9,51.099999,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...
33128,100.0,100.0,900.000004,0.0,43441.027778,3.41,28.4,28.2,84.9,24.5,0.000000,0.0
33129,100.0,100.0,965.000000,0.0,43441.031250,3.26,27.8,27.7,85.5,24.7,0.000000,0.0
33130,100.0,100.0,1169.999989,0.0,43441.034722,3.25,27.6,27.5,85.7,24.6,0.000000,0.0
33131,100.0,100.0,1001.000023,0.0,43441.038194,3.32,27.5,27.2,85.3,24.7,0.000000,0.0


In [118]:
df2

,time,AbsHumOut,Iglob,PARout,Pyrgeo,RadSum,Rain,Rhout,Tout,Winddir,Windsp
0,8/14/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8/14/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8/14/2018,13.369189,0.0,0.0,-32.000000,0.0,0.000000e+00,87.8,17.7,32.0,3.2
3,8/14/2018,13.389380,0.0,0.0,-36.000000,0.0,0.000000e+00,87.4,17.8,32.0,3.2
4,8/14/2018,13.455197,0.0,1.0,-38.999999,0.0,0.000000e+00,87.3,17.9,32.0,3.2
...,...,...,...,...,...,...,...,...,...,...,...
33128,12/7/2018,10.093596,0.0,0.0,-3.000000,0.0,1.000000e+00,96.7,11.6,32.0,9.8
33129,12/7/2018,10.093596,0.0,0.0,-2.000000,0.0,1.000000e+00,96.7,11.6,32.0,10.1
33130,12/7/2018,10.019657,0.0,0.0,-3.000000,0.0,9.999999e-01,96.6,11.5,32.0,10.1
33131,12/7/2018,10.083210,0.0,0.0,-2.000000,0.0,1.340000e-07,96.6,11.6,32.0,10.1


,GHtime,CO2air
0,43326.000000,NaN
1,43326.003472,NaN
2,43326.006944,427.000000
3,43326.010417,443.000000
4,43326.013889,443.999999
...,...,...
33128,43441.027778,900.000004
33129,43441.031250,965.000000
33130,43441.034722,1169.999989
33131,43441.038194,1001.000023


In [119]:
df=pd.DataFrame()
df[['GHtime','CO2air','Tair','VentLee','Ventwind']]=df1[['GHtime','CO2air','Tair','VentLee','Ventwind']]
df[['Tout','Windsp']]=df2[['Tout','Windsp']]
df['GHtime']=pd.TimedeltaIndex(df['GHtime'],unit='d')+dt.datetime(1899,12,30)
#df
#df=df.dropna()
(df['GHtime'][1]-df['GHtime'][0]).total_seconds()
#for i in range(3,len(df['GHtime'])):
#    if ((df['GHtime'][i]-df['GHtime'][i-1]).total_seconds()>500)==True:
#        print(i)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
type((df['GHtime'][2]-df['GHtime'][1]).total_seconds())

float

Tham số đã biết

In [120]:
cap_CO2_Air=3.8 #m
cap_CO2_Top=0.4 #m 
A_Flr=14000 #m^2
U_Blow=1
Eta_HeatCO2=0.057 #mg{CO2}*J^(-1)
P_Blow=500 #W
U_ExtCO2=1
Phi_ExtCO2=72000 #mg*s^(-1)
CO2_Out=668 #mg*m^(-3)
U_Pad=1
Phi_Pad=16.7 #? #m^3*s^(-1)
U_ThScr=1
K_ThScr=0.05*(10**(-3)) #m*K^(-2/3)*s^(-1)
g=9.81 #m*s^(-1)
Delta_T_AirTop= 5 #? #K
RO_Mean= 0.2*(10**3) #kg*m^(-3) #lấy đại ở thermal
Delta_RO=0.5*(10**3) #? #kg*m^(-3)
U_Roof=1
U_Side=1
A_Roof=1400 #? #m^2
A_Side=0.09#? #m^2
C_d=0.2 #?  #Prediction_of_the_Effect_of_Insect-proof_Screens_o.pdf
C_w=0.3 #?
h_SideRoof=1.9 #m(Chua chac chan)
h_Roof=0.68 #m
Delta_T_AirOut= 8 #? #K
T_mean=293#? #K
Zeta_InsScr=1 
C_leakage=1*(10**(-4))
Eta_SideThr=0.9 #? 
Eta_RoofThr=0.9 
U_VentForced=1
Phi_VentForced= 16.7 #? #m^3*s^(-1)
C_Max_Buf=20*(10**3)  #mg{CO2}*m^(-2)
C_Buf=10*(10**3) #? #mg{CO2}*m^(-2)
M_CH2O=30
#Quang hợp
K_T0=0.84
T0=298.15
Ha=37000
R=8.314472
Hd=220000
S=710
Res=250#200
L_0_5=500 #406.5
L0=2000 #545.45
K=1
m=0.1
CO2_0_5=10000 #20000
P_MLT=50 #35
LAI=2

In [121]:
df

,GHtime,CO2air,Tair,VentLee,Ventwind,Tout,Windsp
0,2018-08-14 00:10:00.000048000,427.000000,19.9,52.900000,2.9,17.7,3.2
1,2018-08-14 00:15:00.000028799,443.000000,19.9,52.900000,2.9,17.8,3.2
2,2018-08-14 00:20:00.000009600,443.999999,19.9,51.099999,2.9,17.9,3.2
3,2018-08-14 00:24:59.999990399,435.000001,19.7,42.800001,2.9,17.8,3.2
4,2018-08-14 00:29:59.999971200,442.000000,19.9,42.800000,2.9,17.8,3.2
...,...,...,...,...,...,...,...
32985,2018-12-07 00:34:59.999952000,952.999999,24.5,0.000000,0.0,11.6,9.8
32986,2018-12-07 00:40:00.000019200,900.000004,24.5,0.000000,0.0,11.6,9.8
32987,2018-12-07 00:45:00.000000000,965.000000,24.7,0.000000,0.0,11.6,10.1
32988,2018-12-07 00:49:59.999980799,1169.999989,24.6,0.000000,0.0,11.5,10.1


In [122]:
MC_BlowAir=0
MC_ExtAir=0
MC_PadAir=0
MC_AirCan=0
MC_AirTop=0
MC_AirOut=0
MC_TopOut=0
CO2_Air=0
CO2_Top=0
T=0
Eta_Side=0
Eta_Roof=0
v_Wind=0

In [123]:

for i in range(0,1):
    #KHAI Báo biến thay đổi dựa vào DATA
    CO2_Air=float(df['CO2air'][i]*0.0409*44.01)
    CO2_Top=CO2_Air
    T=float(df['Tout'][i]+273)
    Eta_Side=float(df['Ventwind'][i]/100)
    Eta_Roof=float(df['VentLee'][i]/100)
    v_Wind=float(df['Windsp'][i])
    print(dx(cap_CO2_Air,cap_CO2_Top,MC_BlowAir,MC_ExtAir,MC_PadAir,MC_AirCan,MC_AirTop,MC_AirOut,MC_TopOut))


3.2
0.0020357142857142857
[(1.2864923316028705-334.15061600511876j), -4.513344673570341]


Chú ý biến trùng tên